# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f69f4555280>
├── 'a' --> tensor([[ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248]])
└── 'x' --> <FastTreeValue 0x7f69f4555040>
    └── 'c' --> tensor([[ 0.6849, -0.6786, -1.2493, -1.3095],
                        [ 0.0337, -0.7136,  0.5151,  1.6843],
                        [-0.5719, -0.4093, -0.3818,  1.1745]])

In [4]:
t.a

tensor([[ 0.8343,  0.4898, -1.5337],
        [-1.6319,  0.7745,  1.4248]])

In [5]:
%timeit t.a

76 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f69f4555280>
├── 'a' --> tensor([[-0.4342,  0.4469, -1.0358],
│                   [-0.7354, -0.9062, -0.7942]])
└── 'x' --> <FastTreeValue 0x7f69f4555040>
    └── 'c' --> tensor([[ 0.6849, -0.6786, -1.2493, -1.3095],
                        [ 0.0337, -0.7136,  0.5151,  1.6843],
                        [-0.5719, -0.4093, -0.3818,  1.1745]])

In [7]:
%timeit t.a = new_value

79.7 ns ± 1.65 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248]]),
    x: Batch(
           c: tensor([[ 0.6849, -0.6786, -1.2493, -1.3095],
                      [ 0.0337, -0.7136,  0.5151,  1.6843],
                      [-0.5719, -0.4093, -0.3818,  1.1745]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8343,  0.4898, -1.5337],
        [-1.6319,  0.7745,  1.4248]])

In [11]:
%timeit b.a

67.7 ns ± 0.92 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8528,  1.0403, -0.3512],
               [ 0.3105, -0.7269,  0.8902]]),
    x: Batch(
           c: tensor([[ 0.6849, -0.6786, -1.2493, -1.3095],
                      [ 0.0337, -0.7136,  0.5151,  1.6843],
                      [-0.5719, -0.4093, -0.3818,  1.1745]]),
       ),
)

In [13]:
%timeit b.a = new_value

626 ns ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

975 ns ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

289 µs ± 5.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

272 µs ± 8.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f69f4555100>
├── 'a' --> tensor([[[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]],
│           
│                   [[ 0.8343,  0.4898, -1.5337],
│                    [-1.6319,  0.7745,  1.4248]]])
└── 'x' --> <FastTreeValue 0x7f69f4555220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.9 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f69f4559430>
├── 'a' --> tensor([[ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248],
│                   [ 0.8343,  0.4898, -1.5337],
│                   [-1.6319,  0.7745,  1.4248]])
└── 'x' --> <FastTreeValue 0x7f69f4559b20>
    └── 'c' --> tensor([[ 0.6849, -0.6786, -1.2493, -1.3095],
                        [ 0.0337, -0.7136,  0.5151,  1.6843],
                 

In [23]:
%timeit t_cat(trees)

42.2 µs ± 828 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.8 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]],
       
               [[ 0.8343,  0.4898, -1.5337],
                [-1.6319,  0.7745,  1.4248]]]),
    x: Batch(
           c: tensor([[[ 0.6849, -0.6786, -1.2493, -1.3095],
                       [ 0.0337, -0.7136,  0.5151,  1.6843],
                       [-0.5719, -0.4093, -0.3818,  1.1745]],
         

In [26]:
%timeit Batch.stack(batches)

105 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248],
               [ 0.8343,  0.4898, -1.5337],
               [-1.6319,  0.7745,  1.4248]]),
    x: Batch(
           c: tensor([[ 0.6849, -0.6786, -1.2493, -1.3095],
                      [ 0.0337, -0.7136,  0.5151,  1.6843],
                      [-0.5719, -0.4093, -0.3818,  1.1745],
                      [ 0.6849, -0.6786, -1.2493, -1.3095],
                      [ 0.0337, -

In [28]:
%timeit Batch.cat(batches)

185 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

591 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
